In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utilsforecast.losses import *
from utilsforecast.evaluation import evaluate

from dotenv import load_dotenv
from nixtla import NixtlaClient

warnings.filterwarnings("ignore")
os.environ["NIXTLA_ID_AS_COL"] = "true"
pd.set_option('display.precision', 3)

In [ ]:
plt.rcParams['figure.figsize'] = (9,6)

In [ ]:
url = "https://raw.githubusercontent.com/marcopeix/TimeSeriesForecastingUsingFoundationModels/refs/heads/main/data/walmart_sales_small.csv"

df = pd.read_csv(url, parse_dates=["Date"])
df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['Store'] == i+1]

    ax.plot(plot_df['Date'], plot_df['Weekly_Sales'])
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

# TimeGPT

In [ ]:
load_dotenv()
base_url = os.getenv("BASE_URL")
api_key = os.getenv("NIXTLA_API_KEY")

In [ ]:
# Initialize Nixtla client

## Zero-shot forecasting

In [ ]:
# Zero-shot forecasting

preds.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['Store'] == i+1]
    plot_preds = preds[preds['Store'] == i+1]

    ax.plot(plot_df['Date'], plot_df['Weekly_Sales'])
    ax.plot(plot_preds['Date'], plot_preds['TimeGPT'], color='green', ls='--', label='TimeGPT')
    ax.fill_between(plot_preds['Date'], plot_preds['TimeGPT-hi-80'], plot_preds['TimeGPT-lo-80'], color='green', alpha=0.2)
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')
    ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

## Cross-validation

In [ ]:
# Cross-validation

cv_df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['Store'] == i+1]
    plot_cv = cv_df[cv_df['Store'] == i+1]

    ax.plot(plot_df['Date'], plot_df['Weekly_Sales'])
    ax.plot(plot_cv['Date'], plot_cv['TimeGPT'], color='green', ls='--', label='TimeGPT')
    ax.fill_between(plot_cv['Date'], plot_cv['TimeGPT-hi-80'], plot_cv['TimeGPT-lo-80'], color='green', alpha=0.2)
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')
    ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

## Fine-tuning

In [ ]:
store1_df = df[df['Store'] == 1]

test_size = 32

train = store1_df[:-test_size]
test = store1_df[-test_size:]

print(len(train), len(test))

In [ ]:
baseline_preds = nixtla_client.forecast(
    df=train,
    h=test_size,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales'
)

In [ ]:
longhorizon_preds = nixtla_client.forecast(
    df=train,
    h=test_size,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    # Specify model
)

In [ ]:
test = test.copy()
test = test[['Store', 'Date', 'Weekly_Sales']]
test.loc[:, 'TimeGPT'] = baseline_preds['TimeGPT'].values
test.loc[:, 'TimeGPT-long'] = longhorizon_preds['TimeGPT'].values
test.head()

In [ ]:
evaluation = evaluate(
    df=test,
    metrics=[mae, smape],
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales'
)
evaluation

In [ ]:
longhorizon_finetune_preds = nixtla_client.forecast(
    df=train,
    h=test_size,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    model='timegpt-1-long-horizon',
    # Finetune steps and loss
)
test.loc[:, 'TimeGPT-long-finetuned'] = longhorizon_finetune_preds['TimeGPT'].values

In [ ]:
evaluation = evaluate(
    df=test,
    metrics=[mae, smape],
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales'
)
evaluation

In [ ]:
longhorizon_finetune_preds2 = nixtla_client.forecast(
    df=train,
    h=test_size,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    model='timegpt-1-long-horizon',
    finetune_steps=10,
    finetune_loss='mae',
    # Finetune depth
)
test.loc[:, 'TimeGPT-long-finetuned2'] = longhorizon_finetune_preds2['TimeGPT'].values

In [ ]:
evaluation = evaluate(
    df=test,
    metrics=[mae, smape],
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales'
)
evaluation

In [ ]:
fig, ax = plt.subplots()

ax.plot(store1_df['Date'], store1_df['Weekly_Sales'])
ax.plot(test['Date'], test['Weekly_Sales'])
ax.plot(test['Date'], test['TimeGPT-long-finetuned2'], color='green', ls='--', label='TimeGPT')
ax.set_xlabel('Time')
ax.set_ylabel('Sales')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

# Moirai

In [ ]:
import torch
from gluonts.dataset.pandas import PandasDataset

from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

In [ ]:
df = df[['Store', 'Date', 'Weekly_Sales']]
df = df.set_index('Date')
df.head()

In [ ]:
ds = PandasDataset.from_long_dataframe(df, target='Weekly_Sales', item_id='Store')

In [ ]:
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained("Salesforce/moirai-1.0-R-small"),
    prediction_length=8,
    context_length=len(df.query('Store == 1')),
    patch_size='auto',
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real
)

In [ ]:
# Make predictions


In [ ]:
def get_median_and_ci(
    data, 
    start_date,
    horizon,
    freq,
    id,
    confidence=0.95
):    
    # Calculate the median for each timestep
    medians = np.median(data, axis=0)
    
    # Calculate the lower and upper percentile for the given confidence interval
    lower_percentile = (1 - confidence) / 2 * 100
    upper_percentile = (1 + confidence) / 2 * 100
    
    # Calculate the lower and upper bounds for each timestep
    lower_bounds = np.percentile(data, lower_percentile, axis=0)
    upper_bounds = np.percentile(data, upper_percentile, axis=0)

    pred_dates = pd.date_range(start=start_date, periods=horizon, freq=freq)
    formatted_dates = pred_dates.strftime('%m-%d-%Y').tolist()
    
    # Create a DataFrame with the results
    df = pd.DataFrame({
        'Date': formatted_dates,
        'Store': id,
        'Moirai': medians,
        f'Moirai-lo-{int(confidence*100)}': lower_bounds,
        f'Moirai-hi-{int(confidence*100)}': upper_bounds
    })
    
    return df

In [ ]:
preds = [
    get_median_and_ci(
        data=forecasts[i].samples,
        start_date='11-02-2012',
        horizon=8,
        freq='W-FRI',
        id=i+1,
        confidence=0.80
    )
    for i in range(4)
]

preds_df = pd.concat(preds, axis=0, ignore_index=True)
preds_df['Date'] = pd.to_datetime(preds_df['Date'])

preds_df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

plot_df = df.reset_index(drop=False).copy()
plot_preds_df = preds_df.copy()

for i, ax in enumerate(axes.flatten()):
    df = plot_df[plot_df['Store'] == i+1].iloc[-100:]
    preds_df = plot_preds_df[plot_preds_df['Store'] == i+1]

    ax.plot(df['Date'], df['Weekly_Sales'])
    ax.plot(preds_df['Date'], preds_df['Moirai'], 'g--', label='Moirai')
    ax.fill_between(preds_df['Date'], preds_df['Moirai-lo-80'], preds_df['Moirai-hi-80'], alpha=0.1, color='green')
    ax.legend()
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

## Evaluate Moirai

In [ ]:
df = pd.read_csv(url, parse_dates=["Date"])
df = df[['Store', 'Date', 'Weekly_Sales']]
df = df[df['Store'] == 1]
df_train = df[:-32]
df_train = df.set_index('Date')
ds = PandasDataset.from_long_dataframe(df_train, target='Weekly_Sales', item_id='Store')

In [ ]:
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained("Salesforce/moirai-1.0-R-small"),
    prediction_length=32,
    context_length=111,
    patch_size='auto',
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real
)

In [ ]:
predictor = model.create_predictor(batch_size=32)
forecasts = predictor.predict(ds)
forecasts = list(forecasts)

In [ ]:
moirai_preds = get_median_and_ci(
    data=forecasts[0].samples,
    start_date='03-23-2012',
    horizon=32,
    freq='W-FRI',
    id=1,
    confidence=0.80
)
moirai_preds.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(store1_df['Date'], store1_df['Weekly_Sales'])
ax.plot(test['Date'], test['Weekly_Sales'])
ax.plot(test['Date'], moirai_preds['Moirai'], color='green', ls='--', label='Moirai')
ax.fill_between(test['Date'], moirai_preds['Moirai-lo-80'], moirai_preds['Moirai-hi-80'], color='green', alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Sales')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
test.loc[:, 'Moirai'] = moirai_preds['Moirai'].values
evaluation = evaluate(
    df=test,
    metrics=[mae, smape],
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales'
)
evaluation

# Chronos

In [ ]:
from chronos import BaseChronosPipeline

In [ ]:
df = pd.read_csv(url, parse_dates=["Date"])

In [ ]:
# Create context

In [ ]:
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-small",
    device_map="cpu",  # use "cuda" for GPU inference
    torch_dtype=torch.bfloat16,
)

In [ ]:
# Make predictions


In [ ]:
start_date = pd.to_datetime('2012-10-26')
forecast_dates =  [start_date + pd.DateOffset(weeks=i) for i in range(1, 9)]
preds = mean.numpy()
quantiles = quantiles.numpy()

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(14,8))

for i, ax in enumerate(axes.flatten()):
    store_id = i+1
    data = df.query("Store == @store_id")

    ax.plot(data['Date'], data['Weekly_Sales'])
    ax.plot(forecast_dates, preds[i], ls='--', color='green', label='Chronos')
    ax.fill_between(forecast_dates, quantiles[i, :, 0], quantiles[i, :, 2], color="green", alpha=0.2)
    
    ax.set_title(f"Store {store_id}")
    ax.set_xlabel('Date')
    ax.set_ylabel('Sales volume ($)')
    ax.legend(loc=1)

fig.autofmt_xdate()
plt.tight_layout()

## Evaluate Chronos

In [ ]:
df = df[df['Store'] == 1]
train_df = df[:-32]

In [ ]:
context = torch.tensor(train_df['Weekly_Sales'])

In [ ]:
quantiles, mean = pipeline.predict_quantiles(
    context=context,
    prediction_length=32,
    quantile_levels=[0.1, 0.5, 0.9],
)

In [ ]:
quantiles = quantiles.numpy()
fig, ax = plt.subplots()

ax.plot(store1_df['Date'], store1_df['Weekly_Sales'])
ax.plot(test['Date'], test['Weekly_Sales'])
ax.plot(test['Date'], mean.numpy()[0], ls='--', color='green', label='Chronos')
ax.fill_between(test['Date'], quantiles[0, :, 0], quantiles[0, :, 2], color='green', alpha=0.2)
ax.set_xlabel('Time')
ax.set_ylabel('Sales')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
test.loc[:, 'Chronos'] = mean.numpy()[0]
evaluation = evaluate(
    df=test,
    metrics=[mae, smape],
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales'
)
evaluation